<img src="https://1000logos.net/wp-content/uploads/2017/11/Zillow-Logo.png" title="Zillow Logo"/>

<center><h1><span style="color:#0074E3">Bringing Down the House/Property Value Predictors</span>
</h1>
    <br>by Jennifer Eyring, Jemison Cohort

_________________________________________________________________________________________________________________

 <b>Intro notes:</b>
 In response to the email sent out by the Zillow Data Science team,  I have conducted some thorough research to improve and add-on to the current Zillow property tax value predictor.
<br><br>
This report is a walk-through of the main findings from my data exploration that gives our Zillow Data Science team<br><b> a _______Regression Model that returns a ______-% accurate predicition of what Single Family property tax values are.

_________________________________________________________________________________________________________________

># Project Goals:
><ol>
><li><b>Construct ML Regression model that predicts propery tax assessed values at a higher rate than the current model</b></li>
 <li>To find the key property feature drivers that relate to property value for single family properties.</li>
 <li>To find and add the fipcode state and county names to the 2017 dataframe.</li>
</ol>

In [ ]:
># Project Goals:
><ol>
><li><b>Construct ML Regression model that predicts propery tax assessed values at a higher rate than the current model</b></li>
 <li>To find the key property feature drivers that relate to property value for single family properties.</li>
 <li>To find and add the fipcode state and county names to the 2017 dataframe.</li>
</ol>

<img src="https://play-lh.googleusercontent.com/dKL38bnxOQTTyG9dB467IfCYF0gHWZROJeDUiSGEuVrjaSIHbjDr9T6P6y1OrB_KAQ" title="house" width="100" height="40"/>